In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
features, _ = datasets.make_moons(n_samples=10000, noise=0.05, random_state=1)  # labels we don't care about
features = features.astype(np.float32)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(features[:, 0], features[:, 1], s=2, alpha=0.1);

In [ ]:
from normalizing_flows.one_dimensional import flow_model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
cf = flow_model.CouplingFlow1D(2, device=device)
cf = cf.to(device)

In [ ]:
class MoonsDataset(Dataset):
    def __init__(self, features):
        self.features = torch.from_numpy(features)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx]

# Create dataset and dataloader
dataset = MoonsDataset(features)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Print some information about the dataloader
print(f"Dataset size: {len(dataset)}")
print(f"Number of batches: {len(dataloader)}")
print(f"Batch shape: {next(iter(dataloader)).shape}")

In [ ]:
opt = torch.optim.Adam(cf.parameters(), lr=1e-4)
n_epochs = 10

cf = cf.train()
cf = cf.to(device)
for ep in range(n_epochs):
    loss_sum = 0
    for i, x in enumerate(dataloader):
        x = x.to(device)
        opt.zero_grad()
        loss = -cf.log_prob(x).mean()
        loss.backward()
        opt.step()
        loss_sum += loss.detach().cpu().item()
        # break
    print(f"Epoch {ep+1}/{n_epochs}, loss: {loss_sum / len(dataloader)}")

In [ ]:
sampled_features = cf.sample(1000).cpu().numpy()

fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(sampled_features[:, 0], sampled_features[:, 1], s=2, alpha=0.1);

In [ ]:
next(cf.parameters())